In [187]:
import xmltodict
import numpy as np
import json
import pprint

In [21]:
# % is the comment for SON

In [70]:
f = json.loads(open('m.json').read())

In [71]:
f.keys()

dict_keys(['annotations', 'schema', 'specs'])

In [75]:
f['annotations'][ex].keys()

dict_keys(['all_parents', 'doc', 'entity', 'name', 'niche', 'parents', 'vars'])

In [89]:
f['annotations'][ex]['vars'].keys()

dict_keys(['feed', 'feed_commod_prefs', 'feed_commods', 'feed_recipe', 'feedbuf_size', 'latitude', 'leftover', 'leftover_commod', 'leftoverbuf_size', 'longitude', 'streams', 'streams_', 'throughput'])

In [91]:
print(ex)
for key, val in f['annotations'][ex]['vars'].items():
    print(key)
    pprint(val)
    print('\n\n')

:cycamore:Separations
feed
{'capacity': 'feedbuf_size',
 'index': 4,
 'shape': [-1, -1],
 'type': ['cyclus::toolkit::ResBuf', 'cyclus::Material']}



feed_commod_prefs
{'alias': ['feed_commod_prefs', 'val'],
 'default': [],
 'doc': 'Feed commodity request preferences for each of the given feed '
        'commodities (same order). If unspecified, default is to use 1.0 for '
        'all preferences.',
 'index': 1,
 'shape': [-1, -1],
 'tooltip': ['feed_commod_prefs', ''],
 'type': ['std::vector', 'double'],
 'uilabel': ['Feed Commodity Preference List', '']}



feed_commods
{'alias': ['feed_commods', 'val'],
 'doc': 'Ordered list of commodities on which to request feed material to '
        'separate. Order only matters for matching up with feed commodity '
        'preferences if specified.',
 'index': 0,
 'shape': [-1, -1],
 'tooltip': ['feed_commods', ''],
 'type': ['std::vector', 'std::string'],
 'uilabel': ['Feed Commodity List', ''],
 'uitype': ['oneormore', 'incommodity']}



fee

In [106]:
pprint.pprint(f['annotations'][ex]['vars']['streams_'])

{'alias': [['streams', 'item'],
           'commod',
           ['info', 'buf_size', [['efficiencies', 'item'], 'comp', 'eff']]],
 'doc': 'Output streams for separations. Each stream must have a unique name '
        'identifying the commodity on  which its material is traded, a max '
        'buffer capacity in kg (neg values indicate infinite size), and a set '
        "of component efficiencies. 'comp' is a component to be separated into "
        "the stream (e.g. U, Pu, etc.) and 'eff' is the mass fraction of the "
        'component that is separated from the feed into this output stream. If '
        'any stream buffer is full, the facility halts operation until space '
        'becomes available. The sum total of all component efficiencies across '
        'streams must be less than or equal to 1 (e.g. sum of U efficiencies '
        'for all streams must be <= 1).',
 'index': 9,
 'shape': [-1, -1, -1, -1, -1, -1, -1],
 'tooltip': [['streams_', ''], '', ['', '', [['', ''], '', 

In [103]:
skiplist = ['feed']
def read_annotation(ad, full_name):
    type_ = ad['entity']
    doc = ad['doc']
    name = ad['name']
    callname = full_name.split(':')[-1]
    var_dict = {}
    for key, val in ad['vars'].items():
        print(key)
        # filter out weird dudes
        if type(val) != dict:
            continue
        w = False
        for k in val['type']:
            if 'cyclus' in k:
                w = True
        if w:
            continue
        var_dict[key] = {}
        if type(val['type']) == str:
            var_dict[key]['scalar'] = True
            var_dict[key]['type'] = val['type']
        else:
            var_dict[key]['scalar'] = False
            var_dict[key]['type'] = val['type'][-1]
            var_dict[key]['alias'] = val['alias'][-1]
        
        if 'default' in val.keys():
            var_dict[key]['optional'] = True
        else:
            var_dict[key]['optional'] = False
        var_dict[key]['doc'] = val['doc']
    return var_dict
        
        
        

read_annotation(f['annotations'][ex], ex)

feed
feed_commod_prefs
feed_commods
feed_recipe
feedbuf_size
latitude
leftover
leftover_commod
leftoverbuf_size
longitude
streams
streams_
throughput


{'feed_commod_prefs': {'scalar': False,
  'type': 'double',
  'alias': 'val',
  'optional': True,
  'doc': 'Feed commodity request preferences for each of the given feed commodities (same order). If unspecified, default is to use 1.0 for all preferences.'},
 'feed_commods': {'scalar': False,
  'type': 'std::string',
  'alias': 'val',
  'optional': False,
  'doc': 'Ordered list of commodities on which to request feed material to separate. Order only matters for matching up with feed commodity preferences if specified.'},
 'feed_recipe': {'scalar': True,
  'type': 'std::string',
  'optional': True,
  'doc': 'Name for recipe to be used in feed requests. Empty string results in use of a dummy recipe.'},
 'feedbuf_size': {'scalar': True,
  'type': 'double',
  'optional': False,
  'doc': 'Maximum amount of feed material to keep on hand.'},
 'latitude': {'scalar': True,
  'type': 'double',
  'optional': True,
  'doc': "Latitude of the agent's geographical position. The value should be express

In [109]:
specs = f['specs']
ex = specs[-4]


#def read_schema()
xmltodict.parse(f['schema'][ex])['interleave'].keys()
d = xmltodict.parse(f['schema'][ex])

In [178]:
conversion_dict = {'string': 'String',
                   'nonNegativeInteger': 'Int',
                   'boolean': 'Int',
                   'double': 'Real',
                   'positiveInteger': 'Int',
                   'float': 'Real',
                   'double': 'Real',
                   'duration': 'Int',
                   'integer': 'Int',
                   'nonPositiveInteger': 'Int',
                   'negativeInteger': 'Int',
                   'long': 'Real',
                   'int': 'Int',
                   }
    
def read_element(eld, from_one_or_more=False, optional=False):
    if 'interleave' in eld.keys():
        s = read_interleave(eld['interleave'], eld['@name'], from_one_or_more, optional)
        return s

    # now there's optional and non-optional
    keys = eld.keys()
    if not from_one_or_more:
        options = {'MaxOccurs': 1}
    else:
        options = {}
    if not optional:
        options['MinOccurs'] = 1
        
    s = {eld['@name']: options}
    www = np.random.uniform(0, 10)
    if 'oneOrMore' in keys:

        # s = {eld['@name']: {}}
        s[eld['@name']].update(read_element(eld['oneOrMore']['element'],
                                       from_one_or_more=True)
                              )

       
        return s

    if 'data' in keys:
        options['ValType'] = conversion_dict[eld['data']['@type']]
        s[eld['@name']] = options
        return s

            
def read_interleave(intd, name, from_one_or_more, optional):
    if not optional:
        options = {'MinOccurs':1}
    else:
        options = {}
    if not from_one_or_more:
        options['MaxOccurs'] = 1
    d = {name: options}
    for i in intd['element']:
        d[name].update(read_element(i))
    return d



sep_dict = {}
for i in json.loads(json.dumps(d['interleave']['element'])):
    sep_dict.update(read_element(i))
for j in json.loads(json.dumps(d['interleave']['optional'])):
    sep_dict.update(read_element(j['element'], optional=True))

In [193]:
s = pprint.pformat(sep_dict)
print(s)

{'feed_commod_prefs': {'MaxOccurs': 1,
                       'val': {'MinOccurs': 1, 'ValType': 'Real'}},
 'feed_commods': {'MaxOccurs': 1,
                  'MinOccurs': 1,
                  'val': {'MinOccurs': 1, 'ValType': 'String'}},
 'feed_recipe': {'MaxOccurs': 1, 'ValType': 'String'},
 'feedbuf_size': {'MaxOccurs': 1, 'MinOccurs': 1, 'ValType': 'Real'},
 'latitude': {'MaxOccurs': 1, 'ValType': 'Real'},
 'leftover_commod': {'MaxOccurs': 1, 'ValType': 'String'},
 'leftoverbuf_size': {'MaxOccurs': 1, 'ValType': 'Real'},
 'longitude': {'MaxOccurs': 1, 'ValType': 'Real'},
 'streams': {'MaxOccurs': 1,
             'MinOccurs': 1,
             'item': {'MinOccurs': 1,
                      'commod': {'MaxOccurs': 1,
                                 'MinOccurs': 1,
                                 'ValType': 'String'},
                      'info': {'MaxOccurs': 1,
                               'MinOccurs': 1,
                               'buf_size': {'MaxOccurs': 1,
              

In [198]:
def schema_dict_string_to_template(d):
    s = pprint.pformat(delete_keys_from_dict(d, ['MaxOccurs', 'MinOccurs', 'ValType']))
    print(s)
    s = s[1:]
    s = '\n'.join(s.split('\n'))
    s = s[:-1]
    s = s.replace("'", '')
    s = s.replace(':', '')
    
    s = s.replace('"', '')
    return s
    
print(schema_dict_string_to_template(sep_dict))


def delete_keys_from_dict(dict_del, lst_keys):
    for k in lst_keys:
        try:
            del dict_del[k]
        except KeyError:
            pass
    for v in dict_del.values():
        if isinstance(v, dict):
            delete_keys_from_dict(v, lst_keys)

    return dict_del


    
def check_if_list(d):
    # this is because some schemas,
    # if there's only one entry would have
    # a dictionary, while if there's multiple
    # would have a list
    if 'ict' in str(type(element)):
        return [d]
    else:
        return d

{'feed_commod_prefs': {'val': {}},
 'feed_commods': {'val': {}},
 'feed_recipe': {},
 'feedbuf_size': {},
 'latitude': {},
 'leftover_commod': {},
 'leftoverbuf_size': {},
 'longitude': {},
 'streams': {'item': {'commod': {},
                      'info': {'buf_size': {},
                               'efficiencies': {'item': {'comp': {},
                                                         'eff': {}}}}}},
 'throughput': {}}
feed_commod_prefs: {val: {}},
 feed_commods: {val: {}},
 feed_recipe: {},
 feedbuf_size: {},
 latitude: {},
 leftover_commod: {},
 leftoverbuf_size: {},
 longitude: {},
 streams: {item: {commod: {},
                      info: {buf_size: {},
                               efficiencies: {item: {comp: {},
                                                         eff: {}}}}}},
 throughput: {}


{'feed_commods': {'val': {}},
 'feedbuf_size': {},
 'streams': {'item': {'commod': {},
   'info': {'buf_size': {},
    'efficiencies': {'item': {'comp': {}, 'eff': {}}}}}},
 'feed_commod_prefs': {'val': {}},
 'feed_recipe': {},
 'throughput': {},
 'leftover_commod': {},
 'leftoverbuf_size': {},
 'latitude': {},
 'longitude': {}}